# Futures Spreads

## Imports

In [250]:
from collections import namedtuple
from functools import partial
import os
import sqlite3
from typing import Callable, Tuple

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import scipy as sp
from scipy.stats import norm
import quandl
from tqdm.notebook import trange, tqdm

pd.options.plotting.backend = "plotly"

## Global Variables

In [82]:
DATE_FORMAT = "%Y-%m-%d"
FIGURE_SIZE = dict(width=1500, height=800)

## Fetch Data from Quandl

[OptionWorks Futures Options](https://www.quandl.com/data/OWF-OptionWorks-Futures-Options/documentation)

Security   | Specifications
-----------|-------------------------------------------------------------------------------------------------------
CBT_FV_FV |[Five-Year T-Note Futures - Contract Specs](https://www.cmegroup.com/trading/interest-rates/us-treasury/5-year-us-treasury-note_contractSpecs_futures.html)
CBT_TY_TY  | [10-Year T-Note Futures - Contract Specs](https://www.cmegroup.com/trading/interest-rates/us-treasury/10-year-us-treasury-note_contractSpecs_futures.html)
ICE_B_B    | [Brent Crude Futures](https://www.theice.com/products/219/Brent-Crude-Futures)
ICE_G_G    | [Low Sulphur Gasoil Futures](https://www.theice.com/products/34361119/Low-Sulphur-Gasoil-Futures)

In [4]:
quandl.ApiConfig.api_key = os.getenv("QUANDL_API_KEY")

start_date = '2018-12-03'
end_date = '2020-08-31'

months = "HMUZ"
years = ["2019", "2020"]
exps = [f"{m}{y}" for y in years for m in months]
column_index = [1, 15, 16]

securities = [
    ("CBT_FV_FV", 1),
    ("CBT_TY_TY", 1),
    ("ICE_B_B", 1),
    ("ICE_G_G", 1/7.45)
    ]

def get_code(
    sec: str,
    exch: str = "OWF",
    exp: str = "2M",
    ts: str = "IVM"
) -> tuple:
    """Constructs string to retrieve individual security."""

    return (f"{exch}/{sec[0]}_{exp}_{ts}", {"column_index": column_index})

sec_list = [get_code(s, exp=exp) for s in securities for exp in exps]

query_params = {
    "dataset": sec_list,
    "start_date": start_date, "end_date": end_date
}

In [5]:
if True:
    data = quandl.get(**query_params)

## Prepare Data

In [6]:
def split_column(c: str) -> tuple:
    """Split column name into individual data elements."""

    elems = c.split("/")
    feed = elems[0]
    descr, series = elems[1].split(" - ")
    exch, op, fu, exp, model = descr.split("_")
    sec = f"{exch}_{op}_{fu}"
    
    return tuple([feed, sec, exp, model, series])

In [51]:
def expand_series(s: pd.Series) -> pd.DataFrame:
    """Expand original series to include seprate series for each data element."""

    labels = ["data_feed", "security", "expiration", "model", "series"]
    values = split_column(s.name)

    df_exp = pd.DataFrame({"date": s.index.strftime(DATE_FORMAT)})
    for i, label in enumerate(labels):    
        df_exp[label] = values[i]

    df_exp["value"] = s.values
    df_exp["series"] = df_exp["series"].str.lower()

    return df_exp

In [52]:
df_all = pd.DataFrame()
for c in data.columns:
    if c[-2:] not in ["_x", "_y"]:
        df_all = pd.concat([df_all, expand_series(data[c])])

In [53]:
def set_cols(c):
    return c[0] if not c[1] else c[1]

df_g = df_all.groupby(df_all.columns[:-1].to_list()).max().unstack("series").reset_index()
df_g.columns = list(map(set_cols, df_g.columns))
df_g["multiplier"] = df_g["security"].map({s[0]: s[1] for s in securities})
df_g.head()

,date,data_feed,security,expiration,model,dte,dtt,future,multiplier
0,2018-12-03,OWF,CBT_FV_FV,H2019,IVM,81.21,116.0,112.976562,1.0
1,2018-12-03,OWF,CBT_FV_FV,H2020,IVM,NaN,NaN,NaN,1.0
2,2018-12-03,OWF,CBT_FV_FV,M2019,IVM,172.21,207.0,112.976562,1.0
3,2018-12-03,OWF,CBT_FV_FV,M2020,IVM,NaN,NaN,NaN,1.0
4,2018-12-03,OWF,CBT_FV_FV,U2019,IVM,NaN,NaN,NaN,1.0


## Load Data

In [11]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [12]:
table_name = "futures_spreads"
cursor.execute(f"DROP TABLE IF EXISTS {table_name};").fetchone()

chunk_size = 20000
total = len(df_g)
n_chunks = (total // chunk_size + 1)
for i in trange(n_chunks):
    df_g.iloc[i * chunk_size:(i + 1) * chunk_size].to_sql(table_name, conn, method='multi', if_exists='append', index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

## Query Second Month Contracts

In [13]:
df_second = pd.read_sql("""
    SELECT date, security, expiration, dtt, future * multiplier as adj_future FROM (
        SELECT *, rank() OVER w rank
        FROM futures_spreads
        WHERE dtt > 30
        WINDOW w AS (PARTITION BY security, date ORDER BY dte)
    ) WHERE rank = 1
""", conn)
df_second["date"] = pd.to_datetime(df_second["date"])
df_second = df_second.set_index(["date", "security"]).unstack("security")
df_second.tail()

expiration                                 dtt                    \
security    CBT_FV_FV CBT_TY_TY ICE_B_B ICE_G_G CBT_FV_FV CBT_TY_TY ICE_B_B   
date                                                                          
2020-08-25      Z2020     Z2020   Z2020   Z2020     128.0     118.0    66.0   
2020-08-26      Z2020     Z2020   Z2020   Z2020     127.0     117.0    65.0   
2020-08-27      Z2020     Z2020   Z2020   Z2020     126.0     116.0    64.0   
2020-08-28      Z2020     Z2020   Z2020   Z2020     125.0     115.0    63.0   
2020-08-31      Z2020     Z2020   Z2020   Z2020     122.0     112.0    60.0   

                    adj_future                                 
security   ICE_G_G   CBT_FV_FV   CBT_TY_TY ICE_B_B    ICE_G_G  
date                                                           
2020-08-25   107.0  125.882812  139.171875   46.66  52.516779  
2020-08-26   106.0  125.867188  139.171875   46.58  51.744966  
2020-08-27   105.0  125.781250  138.828125   46.01  50.771812  
2020-08-28   104.0  125.992188  139.093750   46.25  51.241611  
2020-08-31   101.0  126.031250  139.250000   45.66  50.738255

### Test Pairs

In [14]:
test_pairs = [
    ("ICE_B_B", "ICE_G_G", "2019-06-27", 14.799799),
    ("ICE_B_B", "ICE_G_G", "2019-08-13", 16.264966),
    ("CBT_FV_FV", "CBT_TY_TY", "2019-08-16", 11.421875),
    ("CBT_FV_FV", "CBT_TY_TY", "2019-08-23", 11.710938),
]
    
for p in test_pairs:
    s1, s2, date, value = p
    calc = df_second.loc[date, ("adj_future", s2)] - df_second.loc[date, ("adj_future", s1)]
    assert(abs(calc-value) < 1e-4), "Back to the tower Quasimodo..."

print("Money!")

Money!


## Analysis

In [285]:
pairs = [
    ("CBT_FV_FV", "CBT_TY_TY"),
    ("ICE_B_B", "ICE_G_G")
]

def get_spread_label(pair: tuple) -> str:
    return f"{pair[0].split('_')[0]}:{pair[1].split('_')[-1]}-{pair[0].split('_')[-1]}"

def get_spread(
    pair: tuple,
    df: pd.DataFrame,
    label_fn: Callable = get_spread_label,
    price_col: str = "adj_future",
) -> pd.Series:
    """Calculates spread between two series. The spread will be expressed
    as the second security of the pair less the first one.

    Args:
        pair: Tuple of two str elements, one for each security
            in the form `{exchange code}_{futures code}_{options_code}`.
        df: Pandas dataframe with the price data to be plotted. Assumes series
            are accessible with a tuple of `{(price_col, security)}`.
        label_fn: Function that returns a string label for the new series.
        price_col: Label of column in df by which the prices of the underlying
            securities are accessible.
        

    Returns:
        Pandas series of spreadd.
    """
    
    label = label_fn(pair)
    price_1, price_2 = (df[(price_col, sec)] for sec in pair)
    
    return (price_2 - price_1).rename(label)



df_spreads = pd.concat([get_spread(pair, df_second) for pair in pairs], axis=1)

In [ ]:
df_spreads

Let's start of with a simple plot of each spread and some basic summary statistics.

#### CBT:TY-FY
The TY-FY spread generally increased over the period and appears to be much less volatile than the G-B spread. It rose from \$6.6172 at the beginning of the period to \$13.1016 at the end of the period. There does appear to be a spike leading up to 2020-03-09. A similar spike can be observed around that time in the G-B spread. The mean daily dollar change was \$0.0144 and the mean daily percentage change was \0.0017. The daily dollar standard deviation was \$0.2094 and the standard deviation in daily percentage change terms was 0.0188.

In [140]:
fig = df_spreads.plot(title="Futures Spreads",
              labels=dict(index="time", value="spread", variable="securities"), template="none", **FIGURE_SIZE)
fig.update_yaxes(tickprefix="$")
fig.show()

In [286]:
df_stats = pd.concat([
    df_spreads.describe(), 
    df_spreads.diff().describe(),
    df_spreads.pct_change().describe()
    ], axis=1)
tuples = [(label, col) for label in ["dollars", "diff", "pct_change"] for col in df_spreads.columns]
df_stats.columns = pd.MultiIndex.from_tuples(tuples, names=('label', 'security'))
df_stats 

label        dollars                    diff              pct_change  \
security   CBT:TY-FV     ICE:G-B   CBT:TY-FV     ICE:G-B   CBT:TY-FV   
count     452.000000  450.000000  450.000000  446.000000  452.000000   
mean       10.559372   12.921223    0.014358   -0.005517    0.001708   
std         2.195973    4.710429    0.209355    0.990879    0.018756   
min         6.617188    1.203826   -1.226562   -6.636174   -0.096922   
25%         8.388672    9.611695   -0.085938   -0.577265   -0.008617   
50%        10.652344   14.836208    0.015625   -0.037752    0.001459   
75%        12.867188   16.181577    0.125000    0.612987    0.011954   
max        14.187500   20.493221    1.062500    3.225034    0.087855   

label                 
security     ICE:G-B  
count     452.000000  
mean        0.007518  
std         0.146384  
min        -0.609314  
25%        -0.043258  
50%        -0.002759  
75%         0.047999  
max         1.053632

* Side by side underlying asset with spreads
* Side by side rolling averages versus original spread
* Median and standard deviation of difference to N-day rolling average
* Scatter plot of daily returns with normal distribution and kurtosis stats

In [287]:
def make_spread_charts(
    pairs: tuple,
    df: pd.DataFrame,
    title_text: str,
    price_col: str = "adj_future"
) -> go.Figure:
    """Returns figure for side-by-side charts of the prices of the underlying
    securities and spreads. The spread will be expressed as the second security
    of the pair less the first one with the second security appearing in top row.

    Args:
        pairs: Tuple of two tuples with two str elements, one for each security
            in the form `{exchange code}_{futures code}_{options_code}`.
        df: Pandas dataframe with the price data to be plotted. Assumes series
            are accessible with a tuple of `{(price_col, security)}`.
        title_text: Text of overall figure.
        price_col: Label of column in df by which the prices of the underlying
            securities are accessible.

    Returns:
        A plotly Figure ready for plotting

    """

    subplot_titles = (pairs[0][1], pairs[1][1], pairs[0][0], pairs[1][0])
    subplot_titles += tuple(get_spread_label(p) for p in pairs)

    fig = make_subplots(rows=3, cols=2, subplot_titles=subplot_titles)

    y_ranges = {}

    for i, pair in enumerate(pairs):
        for j, security in enumerate(pair):
            series = df[(price_col, security)]
            fig.append_trace(go.Scatter(
                x=df.index,
                y=series,
                name=security
            ), row=2-j, col=2-i)

            y_ranges[(2-j, 2-i)] = (series.max() - series.min(), series.min(), series.max())
        
        fig.append_trace(go.Scatter(
            x=df.index,
            y=get_spread(pair, df=df, price_col=price_col),
            name=get_spread_label(pair)
        ), row=3, col=i+1)
    
    title_text = f"{title_text}: {df.index.min().strftime(DATE_FORMAT)} - {df.index.max().strftime(DATE_FORMAT)}"
    fig.update_layout(template="none", **FIGURE_SIZE, title_text=title_text, showlegend=False)
    fig.update_yaxes(tickprefix="$")

    # This is hacky, but makes it so the y ranges of the underlying securities in a pair are the same
    # to make it easier to see which movements are causing the spread to change.
    for c in range(1, 3):
        delta = y_ranges[(1, c)][0] - y_ranges[(2, c)][0]
        if delta > 0:
            new_range = (y_ranges[(2, c)][1] - delta / 2, y_ranges[(2, c)][2] + delta / 2)
            fig.update_yaxes(range=new_range, row=2, col=c)
        else:
            new_range = (y_rangess[(1, c)][1] + delta / 2, y_ranges[(1, c)][2] - delta / 2)
            fig.update_yaxes(range=new_range, row=1, col=c)

    return fig


fig = make_spread_charts(pairs, df_second, title_text="2nd Month Futures Spreads")
fig.show()

In [281]:
def make_tail_charts(
    pairs: tuple,
    df: pd.DataFrame,
    title_text: str,
    price_col: str = "adj_future"
) -> go.Figure:
    """Returns figure for side-by-side scatter plots of daily returns overlayed on
    the normal distribution with kurtosis statistics.

    Args:
        pairs: Tuple of two tuples with two str elements, one for each security
            in the pair of the form `{exchange code}_{futures code}_{options_code}`.
        df: Pandas dataframe with the price data to be plotted. Assumes series
            are accessible with a tuple of `{(price_col, security)}`.
        title_text: Text of overall figure.
        price_col: Label of column in df by which the prices of the underlying
            securities are accessible.

    Returns:
        A plotly Figure ready for plotting

    """

    subplot_titles = tuple(get_spread_label(p) for p in pairs)

    fig = make_subplots(rows=1, cols=2, subplot_titles=subplot_titles)

    for i, pair in enumerate(pairs):

        spread = get_spread(pair, df=df, price_col=price_col)
        returns = pd.cut(spread.pct_change(), 100).value_counts().sort_index()
        midpoints = returns.index.map(lambda interval: interval.right).to_numpy()
        norm_dist = norm.pdf(midpoints, loc=spread.pct_change().mean(), scale=spread.pct_change().std())

        fig.append_trace(go.Scatter(
            x=[interval.mid for interval in returns.index],
            y=(returns / returns.sum()) * 100,
            name=spread.name
        ), row=1, col=i+1)

        fig.append_trace(go.Scatter(
            x=[interval.mid for interval in returns.index],
            y=(norm_dist / norm_dist.sum()) * 100,
            name="Normal Distribution"
        ), row=1, col=i+1)
        
    title_text = f"{title_text}: {df.index.min().strftime(DATE_FORMAT)} - {df.index.max().strftime(DATE_FORMAT)}"
    fig.update_layout(template="none", width=FIGURE_SIZE["width"], height=500, title_text=title_text, showlegend=False)
    fig.update_yaxes(tickprefix="%")

    for d in fig["data"]:
        xaxis = d["xaxis"]
        xaxis = f"xaxis{xaxis.replace('x', '')}"
        tickvals = d["x"][::3]
        fig.update_layout({xaxis: {"tickmode": "array", "tickvals": tickvals, "tickformat": ".4f"}})
    
    return fig


fig = make_tail_charts(pairs, df_second, title_text="2nd Month Futures Spreads - Distribution of Daily Returns")
fig.show()


What would be interesting here would be to look at the distributions with the outlier event taken out.

In [288]:
spread = get_spread(pairs[0], df=df_second)
returns = pd.cut(spread.pct_change(), 100).value_counts().sort_index()
norm.pdf(returns.index.map(lambda interval: interval.mid).to_numpy(), loc=spread.pct_change().mean(), scale=spread.pct_change().std())

array([2.64885725e-05, 4.53061226e-05, 7.46383943e-05, 1.21770665e-04,
       1.96742448e-04, 3.14795461e-04, 4.98808509e-04, 7.82734082e-04,
       1.21638102e-03, 1.87197492e-03, 2.85302395e-03, 4.30611683e-03,
       6.43637292e-03, 9.42808778e-03, 1.38243015e-02, 2.00741783e-02,
       2.88673792e-02, 4.11104221e-02, 5.79791062e-02, 8.09778226e-02,
       1.12004552e-01, 1.53419391e-01, 2.08113299e-01, 2.79572475e-01,
       3.71932382e-01, 4.90014092e-01, 6.39334512e-01, 8.26081179e-01,
       1.05704215e+00, 1.33948201e+00, 1.68095646e+00, 2.08906062e+00,
       2.57110978e+00, 3.13375614e+00, 3.78255086e+00, 4.52146689e+00,
       5.32873626e+00, 6.24860386e+00, 7.25632556e+00, 8.34498435e+00,
       9.50406272e+00, 1.07193403e+01, 1.19729681e+01, 1.32437384e+01,
       1.45075595e+01, 1.57381299e+01, 1.69077919e+01, 1.79801980e+01,
       1.89506370e+01, 1.97739758e+01, 2.04317881e+01, 2.09084023e+01,
       2.11902747e+01, 2.12680468e+01, 2.11395447e+01, 2.08095591e+01,
      

In [291]:
fig = make_tail_charts(pairs, df_second.loc[:"2020-02-28"].copy(), title_text="2nd Month Futures Spreads - Distribution of Daily Returns")
fig.show()